# Zgradite klasifikacijski model: Slastne azijske in indijske kuhinje


## Razvrščevalniki kuhinj 2

V tej drugi lekciji o razvrščanju bomo raziskali `več načinov` za razvrščanje kategorijskih podatkov. Prav tako se bomo naučili o posledicah izbire enega razvrščevalnika namesto drugega.

### [**Predhodni kviz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Predpogoji**

Predvidevamo, da ste zaključili prejšnje lekcije, saj bomo nadaljevali z nekaterimi koncepti, ki smo jih že obravnavali.

Za to lekcijo bomo potrebovali naslednje pakete:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [zbirka paketov za R](https://www.tidyverse.org/packages), zasnovana za hitrejše, enostavnejše in bolj zabavno podatkovno znanost!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) je okvir [zbirke paketov](https://www.tidymodels.org/packages/) za modeliranje in strojno učenje.

-   `themis`: [paket themis](https://themis.tidymodels.org/) ponuja dodatne korake receptov za obravnavo neuravnoteženih podatkov.

Pakete lahko namestite z naslednjim ukazom:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativno, spodnji skript preveri, ali imate nameščene potrebne pakete za dokončanje tega modula, in jih namesti, če manjkajo.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Zemljevid klasifikacije**

V naši [prejšnji lekciji](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) smo poskušali odgovoriti na vprašanje: kako izbrati med več modeli? V veliki meri je to odvisno od značilnosti podatkov in vrste problema, ki ga želimo rešiti (na primer klasifikacija ali regresija?).

Prej smo se naučili o različnih možnostih, ki jih imate pri klasifikaciji podatkov, z uporabo Microsoftovega pripomočka. Pythonov okvir za strojno učenje, Scikit-learn, ponuja podoben, vendar bolj podroben pripomoček, ki vam lahko dodatno pomaga zožiti izbiro vaših ocenjevalnikov (drugi izraz za klasifikatorje):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Nasvet: [obiščite ta zemljevid na spletu](https://scikit-learn.org/stable/tutorial/machine_learning_map/) in kliknite po poti, da preberete dokumentacijo.
>
> [Referenčna stran Tidymodels](https://www.tidymodels.org/find/parsnip/#models) prav tako ponuja odlično dokumentacijo o različnih vrstah modelov.

### **Načrt** 🗺️

Ta zemljevid je zelo koristen, ko imate jasno predstavo o svojih podatkih, saj lahko 'hodite' po njegovih poteh do odločitve:

-   Imamo \>50 vzorcev

-   Želimo napovedati kategorijo

-   Imamo označene podatke

-   Imamo manj kot 100K vzorcev

-   ✨ Lahko izberemo Linear SVC

-   Če to ne deluje, ker imamo numerične podatke

    -   Lahko poskusimo ✨ KNeighbors Classifier

        -   Če to ne deluje, poskusite ✨ SVC in ✨ Ensemble Classifiers

To je zelo koristna pot, ki ji lahko sledite. Zdaj pa se lotimo dela z uporabo [tidymodels](https://www.tidymodels.org/) okvira za modeliranje: dosledne in prilagodljive zbirke paketov za R, razvitih za spodbujanje dobre statistične prakse 😊.

## 2. Razdelite podatke in obravnavajte neuravnotežen nabor podatkov.

Iz naših prejšnjih lekcij smo se naučili, da obstaja niz skupnih sestavin med našimi kuhinjami. Prav tako je bila precej neenakomerna porazdelitev števila kuhinj.

To bomo obravnavali tako, da:

-   Odstranimo najpogostejše sestavine, ki povzročajo zmedo med različnimi kuhinjami, z uporabo `dplyr::select()`.

-   Uporabimo `recipe`, ki predhodno obdela podatke, da jih pripravi za modeliranje z uporabo algoritma za `over-sampling`.

To smo že obravnavali v prejšnji lekciji, zato bo to enostavno 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Obdelava neuravnoteženih podatkov

Neuravnoteženi podatki pogosto negativno vplivajo na delovanje modela. Veliko modelov deluje najbolje, ko je število opazovanj enako, zato imajo težave z neuravnoteženimi podatki.

Obstajata predvsem dva načina za obravnavo neuravnoteženih podatkovnih nizov:

-   dodajanje opazovanj v manjšinsko skupino: `Prevzorčenje` (ang. Over-sampling), npr. z uporabo algoritma SMOTE, ki sintetično ustvari nove primere manjšinske skupine z uporabo najbližjih sosedov teh primerov.

-   odstranjevanje opazovanj iz večinske skupine: `Podvzorčenje` (ang. Under-sampling)

V prejšnji lekciji smo prikazali, kako obravnavati neuravnotežene podatkovne nize z uporabo `recepta`. Recept si lahko predstavljamo kot načrt, ki opisuje, kateri koraki naj se uporabijo na podatkovnem nizu, da bo pripravljen za analizo podatkov. V našem primeru želimo doseči enakomerno porazdelitev števila naših kulinarik v `učnem naboru`. Pa začnimo!


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Zdaj smo pripravljeni na treniranje modelov 👩‍💻👨‍💻!

## 3. Onkraj multinomnih regresijskih modelov

V prejšnji lekciji smo obravnavali multinomne regresijske modele. Raziskali bomo nekaj bolj prilagodljivih modelov za klasifikacijo.

### Podporni vektorski stroji

V kontekstu klasifikacije so `Podporni vektorski stroji` tehnika strojnega učenja, ki poskuša najti *hiperploskev*, ki "najbolje" ločuje razrede. Poglejmo preprost primer:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ ne ločuje razredov. H2~ jih ločuje, vendar le z majhno razdaljo. H3~ jih ločuje z največjo razdaljo.

#### Linearni klasifikator podpornih vektorjev

Podporni vektorski razvrščanje (SVC) je del družine tehnik strojnega učenja, imenovanih podporni vektorski stroji. Pri SVC je hiperploskev izbrana tako, da pravilno loči `večino` opazovanj v učnem naboru, vendar `lahko napačno razvrsti` nekaj opazovanj. Z dovoljenjem, da so nekateri podatkovni točki na napačni strani, postane SVM bolj odporen na odstopanja, kar omogoča boljšo posplošitev na nove podatke. Parameter, ki uravnava to kršitev, se imenuje `cost` in ima privzeto vrednost 1 (glej `help("svm_poly")`).

Ustvarimo linearni SVC tako, da nastavimo `degree = 1` v polinomskem modelu SVM.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Zdaj, ko smo zajeli korake predobdelave in specifikacijo modela v *workflow*, lahko nadaljujemo s treniranjem linearnega SVC in hkrati ocenimo rezultate. Za merjenje učinkovitosti ustvarimo nabor metrik, ki bo ocenjeval: `natančnost`, `občutljivost`, `pozitivno napovedno vrednost` in `F-mero`.

> `augment()` bo dodal stolpec(-ce) za napovedi k danim podatkom.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Podporni vektorski stroj

Podporni vektorski stroj (SVM) je razširitev podpornega vektorskega klasifikatorja, ki omogoča uporabo nelinearne meje med razredi. V bistvu SVM-ji uporabljajo *trik s kernelom*, da razširijo prostor značilnosti in se prilagodijo nelinearnim odnosom med razredi. Ena izmed priljubljenih in izjemno prilagodljivih funkcij jedra, ki jih uporabljajo SVM-ji, je *funkcija radialne baze.* Poglejmo, kako se bo obnesla na naših podatkih.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Veliko bolje 🤩!

> ✅ Prosimo, poglejte:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> za dodatno branje.

### Razvrščevalniki najbližjih sosedov

Algoritem *K*-najbližjih sosedov (KNN) je metoda, pri kateri se vsaka opazovana vrednost napove na podlagi njene *podobnosti* z drugimi opazovanji.

Poglejmo, kako ga lahko uporabimo na naših podatkih.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Zdi se, da ta model ne deluje najbolje. Verjetno bo izboljšanje delovanja modela mogoče z spremembo argumentov modela (glejte `help("nearest_neighbor")`). Vsekakor poskusite to možnost.

> ✅ Prosimo, poglejte:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> za več informacij o klasifikatorjih *K*-Najbližjih Sosedov.

### Ensemble klasifikatorji

Ensemble algoritmi delujejo tako, da kombinirajo več osnovnih ocenjevalnikov za izdelavo optimalnega modela bodisi z:

`bagging`: uporabo *povprečne funkcije* na zbirki osnovnih modelov

`boosting`: gradnjo zaporedja modelov, ki se medsebojno nadgrajujejo za izboljšanje napovedne zmogljivosti.

Začnimo z uporabo modela Random Forest, ki gradi veliko zbirko odločitvenih dreves in nato uporabi povprečno funkcijo za boljši celotni model.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Odlično delo 👏!

Poskusimo tudi z modelom Boosted Tree.

Boosted Tree opredeljuje metodo ansambla, ki ustvari serijo zaporednih odločitvenih dreves, kjer vsako drevo temelji na rezultatih prejšnjih dreves, da postopoma zmanjša napako. Osredotoča se na uteži napačno razvrščenih elementov in prilagodi prileganje za naslednji klasifikator, da jih popravi.

Obstajajo različni načini za prileganje tega modela (glejte `help("boost_tree")`). V tem primeru bomo prilegali Boosted drevesa prek pogona `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

✅ Prosimo, da si ogledate:

-   [Strojno učenje za družboslovce](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
-   [Praktično strojno učenje z R](https://bradleyboehmke.github.io/HOML/)
-   [Uvod v statistično učenje z aplikacijami v R](https://www.statlearning.com/)
-   <https://algotech.netlify.app/blog/xgboost/> - Raziskuje model AdaBoost, ki je dobra alternativa xgboost.

za več informacij o Ensemble klasifikatorjih.

## 4. Dodatno - primerjava več modelov

V tem laboratoriju smo prilagodili kar nekaj modelov 🙌. Ustvarjanje številnih delovnih tokov iz različnih naborov predprocesorjev in/ali specifikacij modelov ter nato izračunavanje metrik zmogljivosti enega za drugim lahko postane zamudno ali naporno.

Poglejmo, ali lahko to rešimo z ustvarjanjem funkcije, ki prilagodi seznam delovnih tokov na učnem naboru podatkov in nato vrne metrike zmogljivosti na podlagi testnega nabora. Uporabili bomo `map()` in `map_dfr()` iz paketa [purrr](https://purrr.tidyverse.org/), da funkcije uporabimo na vsakem elementu seznama.

> Funkcije [`map()`](https://purrr.tidyverse.org/reference/map.html) vam omogočajo, da nadomestite številne for zanke s kodo, ki je bolj jedrnata in lažja za branje. Najboljše mesto za učenje o funkcijah [`map()`](https://purrr.tidyverse.org/reference/map.html) je [poglavje o iteraciji](http://r4ds.had.co.nz/iteration.html) v R za podatkovno znanost.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) paket omogoča uporabnikom, da ustvarijo in enostavno prilegajo veliko število modelov, vendar je večinoma zasnovan za delo s tehnikami ponovnega vzorčenja, kot je `križno preverjanje`, pristop, ki ga bomo še obravnavali.

## **🚀Izziv**

Vsaka od teh tehnik ima veliko število parametrov, ki jih lahko prilagodite, na primer `cost` pri SVM, `neighbors` pri KNN, `mtry` (naključno izbrani prediktorji) pri Random Forest.

Raziskujte privzete parametre za vsako od teh tehnik in razmislite, kaj bi prilagajanje teh parametrov pomenilo za kakovost modela.

Če želite izvedeti več o določenem modelu in njegovih parametrih, uporabite: `help("model")`, npr. `help("rand_forest")`.

> V praksi običajno *ocenimo* *najboljše vrednosti* teh parametrov tako, da treniramo veliko modelov na `simuliranem naboru podatkov` in merimo, kako dobro se vsi ti modeli obnesejo. Ta proces imenujemo **uglaševanje**.

### [**Kvizi po predavanju**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Pregled in samostojno učenje**

V teh lekcijah je veliko strokovnih izrazov, zato si vzemite trenutek za pregled [tega seznama](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) uporabne terminologije!

#### HVALA:

[`Allison Horst`](https://twitter.com/allison_horst/) za ustvarjanje neverjetnih ilustracij, ki naredijo R bolj prijazen in privlačen. Več ilustracij najdete v njeni [galeriji](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) in [Jen Looper](https://www.twitter.com/jenlooper) za ustvarjanje izvirne različice tega modula v Pythonu ♥️

Veselo učenje,

[Eric](https://twitter.com/ericntay), zlati Microsoft Learn študentski ambasador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Ilustracija @allison_horst</figcaption>



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
